In [520]:
import pandas as pd
from pathlib import Path
import numpy as np
import alpaca_trade_api as tradeapi
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame

from finta import TA
from finta.utils import resample_calendar
import json
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report

# Alpaca for data
import alpaca_trade_api as api
from alpaca_trade_api.rest import TimeFrame

# pandas for analysis
import pandas as pd

# Plotly for charting
import plotly.graph_objects as go
import plotly.express as px

# Set default charting for pandas to plotly
pd.options.plotting.backend = "plotly"

import datetime as dt
import pytz

import os
import requests



In [521]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [522]:
load_dotenv()

True

In [523]:
# Our API keys for Alpaca
API_KEY = os.getenv('ALPACA_API_KEY')

API_SECRET_KEY = os.getenv('ALPACA_SECRET_KEY')

In [524]:
def alpaca_data_import(coin):

    # Load .env environment variables

    # Our API keys for Alpaca
    API_KEY = os.getenv('ALPACA_API_KEY')

    API_SECRET_KEY = os.getenv('ALPACA_SECRET_KEY')

    #today = dt.date.today()
    # symbols we will be looking at
    #btc = "BTCUSD"
    #spy = "SPY"

    # start dates and end dates for backtest
    #start_date = "2020-01-01"
    #end_date = today 
    #end_date = '2023-02-06'

    end_date = (dt.datetime.now(pytz.timezone('US/Eastern'))-dt.timedelta(hours=1)).isoformat()

    start_date = (dt.datetime.now(pytz.timezone('US/Eastern'))-dt.timedelta(days=1080)).isoformat()

    # time frame for backtests
    timeframe = TimeFrame.Day

    # Setup instance of alpaca api
    alpaca = api.REST(API_KEY, API_SECRET_KEY)

    # # # Request historical bar data for SPY and BTC using Alpaca Data API
    # for equities, use .get_bars
    #spy_data = alpaca.get_bars(spy, timeframe, start_date, end_date).df

    # for crypto, use .get_crypto_bars, from multiple exchanges
    #btc_data = alpaca.get_crypto_bars(btc, timeframe, start_date, end_date).df
    df = alpaca.get_crypto_bars(coin, timeframe, start_date, end_date).df

    df = df[df['exchange'] == 'CBSE']

    # display crypto bar data
    display(df)
    #display(spy_data)

    return df

In [525]:
btc_data = alpaca_data_import('BTCUSD')

,exchange,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,,
2020-02-27 06:00:00+00:00,CBSE,8814.90,8899.94,8779.06,8786.54,1819.421690,15034,8848.335058
2020-02-28 06:00:00+00:00,CBSE,8786.53,8813.12,8428.80,8708.89,11572.123138,63193,8624.928084
2020-02-29 06:00:00+00:00,CBSE,8523.33,8671.56,8508.96,8588.26,1926.708790,15954,8585.057963
2020-03-01 06:00:00+00:00,CBSE,8522.30,8656.23,8487.33,8655.51,2329.079350,15138,8596.448236
2020-03-02 06:00:00+00:00,CBSE,8655.50,8973.45,8591.22,8915.00,7887.291834,47416,8824.567644
...,...,...,...,...,...,...,...,...
2023-02-06 06:00:00+00:00,CBSE,22781.29,23163.05,22639.39,22935.08,10397.136521,236952,22910.702025
2023-02-07 06:00:00+00:00,CBSE,22934.51,23451.01,22770.58,23226.12,9926.531595,294835,23091.005413
2023-02-08 06:00:00+00:00,CBSE,23225.45,23239.79,22360.16,22605.13,11523.796942,317759,22801.309970


In [526]:

# #today = dt.date.today()
# # symbols we will be looking at
# btc = "BTCUSD"
# #spy = "SPY"

# # start dates and end dates for backtest
# #start_date = "2020-01-01"
# #end_date = today 
# #end_date = '2023-02-06'

# end_date = (dt.datetime.now(pytz.timezone('US/Eastern'))-dt.timedelta(hours=1)).isoformat()

# start_date = (dt.datetime.now(pytz.timezone('US/Eastern'))-dt.timedelta(days=1080)).isoformat()

# # time frame for backtests
# timeframe = TimeFrame.Day

In [527]:
# # Our API keys for Alpaca
# API_KEY = os.getenv('ALPACA_API_KEY')

# API_SECRET_KEY = os.getenv('ALPACA_SECRET_KEY')

# # Setup instance of alpaca api
# alpaca = api.REST(ALPACA_API_KEY, ALPACA_SECRET_KEY)

# # # # Request historical bar data for SPY and BTC using Alpaca Data API
# # for equities, use .get_bars
# #spy_data = alpaca.get_bars(spy, timeframe, start_date, end_date).df

# # for crypto, use .get_crypto_bars, from multiple exchanges
# btc_data = alpaca.get_crypto_bars(btc, timeframe, start_date, end_date).df

# btc_data = btc_data[btc_data['exchange'] == 'CBSE']

# # display crypto bar data
# display(btc_data)
# #display(spy_data)

In [528]:
def process_data_ohlcv(df):

    ohlcv_df = df.drop(columns=['exchange','trade_count','vwap'])
    #ohlc_df = df.drop(columns=['volume'])
    ichimoku_df = TA.ICHIMOKU(ohlcv_df, tenkan_period= 20, kijun_period= 60, senkou_period= 120, chikou_period= 30)
    #ETH ichimoku_df = TA.ICHIMOKU(ohlcv_df, tenkan_period= 13, kijun_period= 34, senkou_period= 55, chikou_period= 21)

    ichimoku_df = pd.concat([ichimoku_df, ohlcv_df['close']], axis=1)

    return ichimoku_df
#ohlcv_df

#ohlcv_df = resample_calendar(ohlcv_df, '4h')
#ohlcv_df = ohlcv_df.dropna()


In [529]:
ichimoku_df = process_data_ohlcv(btc_data)
#ichimoku_df

In [530]:
def get_signal(df):
    
    df['signal'] = 0
    df["signal"] = np.where((df["TENKAN"] > df["KIJUN"]) &
                                (df["close"] > df["senkou_span_a"]), 1, 0)

    df['entry/exit'] = df['signal'].diff()
    
    df['actual_returns'] = df['close'].pct_change()
    df["strategy_returns"] = df["actual_returns"] * df["signal"].shift() 

    actual_returns = df['actual_returns'].cumsum()
    
    
    return df

In [531]:
ichimoku_signal_df =  get_signal(ichimoku_df)
#ichimoku_signal_df
sell_signals_df = ichimoku_signal_df['entry/exit'] == -1.0
sell_signals_df.tail(100)

timestamp
2022-11-03 05:00:00+00:00    False
2022-11-04 05:00:00+00:00    False
2022-11-05 05:00:00+00:00    False
2022-11-06 05:00:00+00:00    False
2022-11-07 06:00:00+00:00    False
                             ...  
2023-02-06 06:00:00+00:00    False
2023-02-07 06:00:00+00:00    False
2023-02-08 06:00:00+00:00    False
2023-02-09 06:00:00+00:00    False
2023-02-10 06:00:00+00:00    False
Name: entry/exit, Length: 100, dtype: bool

In [532]:
def create_order(symbol, qty, side, order_type, time_in_force, api_key_id, api_secret_key):
    headers = {
        "Apca-Api-Key-Id": api_key_id,
        "Apca-Api-Secret-Key": api_secret_key,
        "Content-Type": "application/json"
    }

    data = {
        "symbol": symbol,
        "qty": qty,
        "side": side,
        "type": order_type,
        "time_in_force": time_in_force
    }

    url = "https://paper-api.alpaca.markets/v2/orders"
    response = requests.post(url, headers=headers, json=data)

    if response.status_code != 200:
        raise ValueError("Failed to create order: " + response.text)

    return response.json()


In [533]:

api_key_id = API_KEY
api_secret_key = API_SECRET_KEY

#result = create_order("BTC/USD", "1.0", "buy", "market", "gtc", api_key_id, api_secret_key)
#print(result)


In [534]:
ichimoku_signal_df['entry/exit'].iloc[-1] 

0.0

In [535]:
if ichimoku_signal_df['entry/exit'].iloc[-1] == 1.0:
    create_order("BTC/USD", "1.0", "buy", "market", "gtc", api_key_id, api_secret_key)
elif ichimoku_signal_df['entry/exit'].iloc[-1] == -1.0:
    create_order("BTC/USD", "1.0", "sell", "market", "gtc", api_key_id, api_secret_key)
else:
    print('No Trades Today!')

No Trades Today!


In [536]:
# ohlc_df = ohlcv_df.drop(columns=['volume'])
# #ohlc_df

In [537]:
# ichimoku_df = TA.ICHIMOKU(ohlcv_df, tenkan_period= 20, kijun_period= 60, senkou_period= 120, chikou_period= 30)
# #ichimoku_df

In [538]:
# ichimoku_signal_df = pd.concat([ichimoku_df, ohlcv_df['close']], axis=1)
ichimoku_signal_df

,TENKAN,KIJUN,senkou_span_a,SENKOU,CHIKOU,close,signal,entry/exit,actual_returns,strategy_returns
timestamp,,,,,,,,,,
2020-02-27 06:00:00+00:00,NaN,NaN,NaN,NaN,6214.81,8786.54,0,NaN,NaN,NaN
2020-02-28 06:00:00+00:00,NaN,NaN,NaN,NaN,6042.61,8708.89,0,0.0,-0.008837,-0.000000
2020-02-29 06:00:00+00:00,NaN,NaN,NaN,NaN,6406.40,8588.26,0,0.0,-0.013851,-0.000000
2020-03-01 06:00:00+00:00,NaN,NaN,NaN,NaN,6347.48,8655.51,0,0.0,0.007830,0.000000
2020-03-02 06:00:00+00:00,NaN,NaN,NaN,NaN,6615.00,8915.00,0,0.0,0.029980,0.000000
...,...,...,...,...,...,...,...,...,...,...
2023-02-06 06:00:00+00:00,22316.095,20267.79,17455.8475,20337.285,NaN,22935.08,1,0.0,0.006724,0.006724
2023-02-07 06:00:00+00:00,22461.135,20267.79,17455.8475,20337.285,NaN,23226.12,1,0.0,0.012690,0.012690
2023-02-08 06:00:00+00:00,22563.910,20267.79,17455.8475,20337.285,NaN,22605.13,1,0.0,-0.026737,-0.026737


In [539]:

# ichimoku_signal_df['signal'] = 0
# ichimoku_signal_df["signal"] = np.where((ichimoku_signal_df["TENKAN"] > ichimoku_signal_df["KIJUN"]) &
#                                 (ichimoku_signal_df["close"] > ichimoku_signal_df["senkou_span_a"]), 1, 0)

# ichimoku_signal_df['entry/exit'] = ichimoku_signal_df['signal'].diff()

# #ichimoku_signal_df


In [540]:
# ichimoku_signal_df['actual_returns'] = ichimoku_signal_df['close'].pct_change()
# ichimoku_signal_df["strategy_returns"] = ichimoku_signal_df["actual_returns"] * ichimoku_signal_df["signal"].shift() 


In [541]:
#ichimoku_signal_df

In [542]:


def get_open_positions(api_key_id, api_secret_key):
    # Initialize the Alpaca API client
   # api = tradeapi.REST(api_key_id, api_secret_key, api_version='v2')
    api = tradeapi.REST(api_key_id, api_secret_key, base_url='https://paper-api.alpaca.markets', api_version='v2')

    # Get a list of all open positions
    open_positions = api.list_positions()
    
    # Return the list of open positions
    
    return open_positions


In [543]:
open_positions = get_open_positions(api_key_id, api_secret_key)


In [544]:
open_positions

[Position({   'asset_class': 'crypto',
     'asset_id': '64bbff51-59d6-4b3c-9351-13ad85e3c752',
     'asset_marginable': False,
     'avg_entry_price': '21927.892204675868242',
     'change_today': '-0.0135751932665143',
     'cost_basis': '1973.5101887813671184',
     'current_price': '21620.38',
     'exchange': 'FTXU',
     'lastday_price': '21917.92',
     'market_value': '1945.8340918981',
     'qty': '0.089999995',
     'qty_available': '0.089999995',
     'side': 'long',
     'symbol': 'BTCUSD',
     'unrealized_intraday_pl': '-26.7785985123',
     'unrealized_intraday_plpc': '-0.0135751932665143',
     'unrealized_pl': '-27.6760968832671184',
     'unrealized_plpc': '-0.0140237922462194'}),
 Position({   'asset_class': 'us_equity',
     'asset_id': '8ccae427-5dd0-45b3-b5fe-7ba5e422c766',
     'asset_marginable': False,
     'avg_entry_price': '143.04',
     'change_today': '-0.0489581323557785',
     'cost_basis': '14304',
     'current_price': '197.17',
     'exchange': 'NASDA

In [545]:
open_positions = get_open_positions(api_key_id, api_secret_key)
open_positions



[Position({   'asset_class': 'crypto',
     'asset_id': '64bbff51-59d6-4b3c-9351-13ad85e3c752',
     'asset_marginable': False,
     'avg_entry_price': '21927.892204675868242',
     'change_today': '-0.0135751932665143',
     'cost_basis': '1973.5101887813671184',
     'current_price': '21620.38',
     'exchange': 'FTXU',
     'lastday_price': '21917.92',
     'market_value': '1945.8340918981',
     'qty': '0.089999995',
     'qty_available': '0.089999995',
     'side': 'long',
     'symbol': 'BTCUSD',
     'unrealized_intraday_pl': '-26.7785985123',
     'unrealized_intraday_plpc': '-0.0135751932665143',
     'unrealized_pl': '-27.6760968832671184',
     'unrealized_plpc': '-0.0140237922462194'}),
 Position({   'asset_class': 'us_equity',
     'asset_id': '8ccae427-5dd0-45b3-b5fe-7ba5e422c766',
     'asset_marginable': False,
     'avg_entry_price': '143.04',
     'change_today': '-0.0489098977426201',
     'cost_basis': '14304',
     'current_price': '197.18',
     'exchange': 'NASDA

In [546]:
#actual_returns = ichimoku_signal_df['actual_returns'].cumsum()

actual_returns_plot =  (1 + ichimoku_signal_df[['actual_returns']]).cumprod().hvplot(
    color='lightblue'
)

In [547]:
strategy_returns_plot = (1 + ichimoku_signal_df[['strategy_returns']]).cumprod().hvplot(
    color='lightgreen'
)

In [548]:
#ichimoku_signal_df

In [549]:
#ichimoku_signal_df['entry/exit'].value_counts()

In [550]:
entry = ichimoku_signal_df[ichimoku_signal_df['entry/exit']==1]['close'].hvplot.scatter(
    color='green',
    marker= '^'
)

exit = ichimoku_signal_df[ichimoku_signal_df['entry/exit']==-1]['close'].hvplot.scatter(
    color='red',
    marker= 'v'
)

close = ichimoku_signal_df['close'].hvplot(
    color='lightgray',
    
)

#actual_returns = actual_returns.hvplot(
 #   color='lightblue'
#)

#strategy_returns = ichimoku_signal_df['strategy_returns'].hvplot(
 #   color='lightgreen',
#)

In [551]:
ichimoku_signal_df['entry/exit'].value_counts()

 0.0    1064
 1.0       8
-1.0       7
Name: entry/exit, dtype: int64

In [552]:
close * entry * exit

:Overlay
   .Curve.Close      :Curve   [timestamp]   (close)
   .Scatter.Close.I  :Scatter   [timestamp]   (close)
   .Scatter.Close.II :Scatter   [timestamp]   (close)

In [553]:
actual_returns_plot * strategy_returns_plot

:Overlay
   .Curve.I  :Curve   [timestamp]   (actual_returns)
   .Curve.II :Curve   [timestamp]   (strategy_returns)

In [554]:
strategy_returns_annual_volitility = ichimoku_signal_df[['strategy_returns']].std()*np.sqrt(365)
strategy_returns_annual_volitility



strategy_returns    0.456349
dtype: float64

In [555]:
annualized_return = ichimoku_signal_df["strategy_returns"].mean() * 365
annualized_std = ichimoku_signal_df["strategy_returns"].std() * np.sqrt(365)
sharpe_ratio = round(annualized_return/annualized_std, 3)


display(annualized_return)
display(annualized_std)
display(sharpe_ratio)

0.6054668016354536

0.45634878283325636

1.327

In [556]:
# ichimoku_df.hvplot()

In [557]:
from pytrends.request import TrendReq

pytrends = TrendReq(hl='en-US', tz=360)

def get_trends(keywords):
    pytrends.build_payload(kw_list=keywords, timeframe='today 12-m')  
    interest_over_time_data = pytrends.interest_over_time()
    return interest_over_time_data

keywords = ['bitcoin']
trends_data = get_trends(keywords)
print(trends_data)


            bitcoin  isPartial
date                          
2022-02-13       44      False
2022-02-20       58      False
2022-02-27       55      False
2022-03-06       48      False
2022-03-13       42      False
2022-03-20       40      False
2022-03-27       43      False
2022-04-03       38      False
2022-04-10       35      False
2022-04-17       33      False
2022-04-24       34      False
2022-05-01       35      False
2022-05-08       80      False
2022-05-15       55      False
2022-05-22       43      False
2022-05-29       38      False
2022-06-05       35      False
2022-06-12      100      False
2022-06-19       73      False
2022-06-26       53      False
2022-07-03       47      False
2022-07-10       43      False
2022-07-17       47      False
2022-07-24       39      False
2022-07-31       35      False
2022-08-07       34      False
2022-08-14       32      False
2022-08-21       31      False
2022-08-28       32      False
2022-09-04       32      False
2022-09-

In [558]:
trends_data_plot =  trends_data['bitcoin'].hvplot(
    color='red',
    
)